# 🤖 HR Agent with Built-in Middleware - Gradio App

This notebook demonstrates LangChain 1.0's built-in middleware with an interactive Gradio interface:

- 📝 **SummarizationMiddleware**: Auto-summarizes long conversations
- 👤 **HumanInTheLoopMiddleware**: Requires approval for critical operations

## Step 1: Install Dependencies

In [ ]:
!pip install -q gradio langchain langchain-openai langgraph

## Step 2: Configure API Key

In [ ]:
import os

# For Google Colab
try:
    from google.colab import userdata
    os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')
    print("✅ API Key loaded from Colab secrets")
except:
    # For local Jupyter - uncomment and add your key:
    # os.environ['OPENAI_API_KEY'] = 'your-api-key-here'
    print("⚠️ Please set your OPENAI_API_KEY")

## Step 3: Import Libraries

In [ ]:
import gradio as gr
from typing import Annotated
import json

from langchain.agents import create_agent
from langchain.agents.middleware import SummarizationMiddleware, HumanInTheLoopMiddleware
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage
from langgraph.checkpoint.memory import InMemorySaver
from langgraph.types import Command

print("✅ All libraries imported")

## Step 4: Define Employee Database

In [ ]:
EMPLOYEES = {
    "101": {"name": "Priya Sharma", "department": "Engineering", "role": "Senior Developer", "salary": 120000, "leave_balance": 12},
    "102": {"name": "Rahul Verma", "department": "Engineering", "role": "Engineering Manager", "salary": 180000, "leave_balance": 8},
    "103": {"name": "Anjali Patel", "department": "HR", "role": "HR Director", "salary": 200000, "leave_balance": 15}
}

print(f"✅ Database loaded: {len(EMPLOYEES)} employees")

## Step 5: Define HR Tools

In [ ]:
@tool
def get_employee_info(employee_id: Annotated[str, "Employee ID"]) -> str:
    """Get employee information by ID."""
    if employee_id in EMPLOYEES:
        emp = EMPLOYEES[employee_id]
        return f"{emp['name']} - {emp['department']} - {emp['role']}"
    return f"Employee {employee_id} not found"

@tool
def check_leave_balance(employee_id: Annotated[str, "Employee ID"]) -> str:
    """Check leave balance for an employee."""
    if employee_id in EMPLOYEES:
        return f"{EMPLOYEES[employee_id]['name']} has {EMPLOYEES[employee_id]['leave_balance']} days remaining"
    return f"Employee {employee_id} not found"

@tool
def update_salary(employee_id: Annotated[str, "Employee ID"], new_salary: Annotated[int, "New salary"]) -> str:
    """Update employee salary. REQUIRES APPROVAL."""
    if employee_id in EMPLOYEES:
        old = EMPLOYEES[employee_id]['salary']
        EMPLOYEES[employee_id]['salary'] = new_salary
        return f"✅ Updated {EMPLOYEES[employee_id]['name']}: ₹{old:,} → ₹{new_salary:,}"
    return f"Employee {employee_id} not found"

@tool
def approve_leave(employee_id: Annotated[str, "Employee ID"], days: Annotated[int, "Days"]) -> str:
    """Approve leave request. REQUIRES APPROVAL."""
    if employee_id in EMPLOYEES:
        emp = EMPLOYEES[employee_id]
        if emp['leave_balance'] >= days:
            EMPLOYEES[employee_id]['leave_balance'] -= days
            return f"✅ Approved {days} days for {emp['name']}. Remaining: {EMPLOYEES[employee_id]['leave_balance']}"
        return f"❌ Insufficient balance: {emp['leave_balance']} days"
    return f"Employee {employee_id} not found"

print("✅ Tools defined")

## Step 6: Create Agent with Middleware

In [ ]:
agent = create_agent(
    model="openai:gpt-4o-mini",
    tools=[get_employee_info, check_leave_balance, update_salary, approve_leave],
    middleware=[
        SummarizationMiddleware(
            model="openai:gpt-4o-mini",
            max_tokens_before_summary=1500,
            messages_to_keep=8,
            summary_prompt="Summarize the HR conversation concisely."
        ),
        HumanInTheLoopMiddleware(
            interrupt_on={
                "update_salary": True,
                "approve_leave": True
            }
        )
    ],
    checkpointer=InMemorySaver(),
    system_prompt="You are an HR assistant. Help with employee info, leave balance, and salary/leave requests (which require approval)."
)

print("✅ Agent created with middleware")

## Step 7: Define Application State

In [ ]:
class AppState:
    def __init__(self):
        self.pending_interrupt = None
        self.current_config = None

state = AppState()
print("✅ State initialized")

## Step 8: Define Handler Functions

In [ ]:
def chat(message, history, thread_id):
    if not message.strip():
        return history, "", None, gr.update(visible=False), ""
    
    config = {"configurable": {"thread_id": thread_id}}
    state.current_config = config
    
    try:
        result = agent.invoke(
            {"messages": [HumanMessage(content=message)]},
            config
        )
        
        if '__interrupt__' in result:
            state.pending_interrupt = result['__interrupt__'][0]
            action = state.pending_interrupt.value['action_requests'][0]
            
            interrupt_msg = f"**🔔 Approval Required**\n\n**Tool:** `{action['name']}`\n**Args:** `{action['arguments']}`"
            args_json = json.dumps(action['arguments'], indent=2)
            
            history.append([message, "⏸️ Operation paused - approval required"])
            return history, "", interrupt_msg, gr.update(visible=True), args_json
        
        response = result['messages'][-1].content
        if any('summary' in str(msg).lower() for msg in result.get('messages', [])[:3]):
            response += "\n\n📝 *Earlier messages were summarized*"
        
        history.append([message, response])
        return history, "", None, gr.update(visible=False), ""
        
    except Exception as e:
        history.append([message, f"❌ Error: {str(e)}"])
        return history, "", None, gr.update(visible=False), ""

def approve():
    if not state.pending_interrupt:
        return "No pending approval", gr.update(visible=False)
    try:
        result = agent.invoke(
            Command(resume={"decisions": [{"type": "approve"}]}),
            state.current_config
        )
        state.pending_interrupt = None
        return f"✅ Approved\n\n{result['messages'][-1].content}", gr.update(visible=False)
    except Exception as e:
        return f"❌ Error: {str(e)}", gr.update(visible=False)

def reject(reason):
    if not state.pending_interrupt:
        return "No pending approval", gr.update(visible=False)
    try:
        result = agent.invoke(
            Command(resume={"decisions": [{"type": "reject", "feedback": reason or "Rejected"}]}),
            state.current_config
        )
        state.pending_interrupt = None
        return f"❌ Rejected: {reason}", gr.update(visible=False)
    except Exception as e:
        return f"❌ Error: {str(e)}", gr.update(visible=False)

def edit_approve(args_json):
    if not state.pending_interrupt:
        return "No pending approval", gr.update(visible=False)
    try:
        args = json.loads(args_json)
        result = agent.invoke(
            Command(resume={"decisions": [{"type": "edit", "args": args}]}),
            state.current_config
        )
        state.pending_interrupt = None
        return f"✏️ Edited & Approved\n\n{result['messages'][-1].content}", gr.update(visible=False)
    except:
        return "❌ Invalid JSON", gr.update(visible=True)

print("✅ Handlers defined")

## Step 9: Create Gradio Interface

In [ ]:
with gr.Blocks(theme=gr.themes.Soft(), title="HR Agent") as demo:
    gr.Markdown("# 🤖 HR Agent with Built-in Middleware\n\n**Try:** Update salary for employee 101 to ₹150,000")
    
    with gr.Row():
        with gr.Column(scale=2):
            chatbot = gr.Chatbot(height=400, label="Chat")
            msg = gr.Textbox(placeholder="Type message...", show_label=False)
            with gr.Row():
                send = gr.Button("Send", variant="primary")
                clear = gr.Button("Clear")
            thread_id = gr.Textbox(value="session_1", label="Thread ID")
        
        with gr.Column(scale=1):
            gr.Markdown("### ✋ Approval Panel")
            approval_box = gr.Column(visible=False)
            with approval_box:
                interrupt_display = gr.Markdown()
                args_editor = gr.Code(language="json", label="Args", lines=4)
                with gr.Row():
                    approve_btn = gr.Button("✅ Approve", size="sm")
                    edit_btn = gr.Button("✏️ Edit", size="sm")
                reject_reason = gr.Textbox(placeholder="Reason", label="Reject")
                reject_btn = gr.Button("❌ Reject", variant="stop")
                result_display = gr.Markdown()
    
    gr.Examples([
        ["Show info for employee 101"],
        ["Update salary for employee 101 to ₹150,000"]
    ], inputs=msg)
    
    send.click(chat, [msg, chatbot, thread_id], [chatbot, msg, interrupt_display, approval_box, args_editor])
    msg.submit(chat, [msg, chatbot, thread_id], [chatbot, msg, interrupt_display, approval_box, args_editor])
    approve_btn.click(approve, None, [result_display, approval_box])
    reject_btn.click(reject, [reject_reason], [result_display, approval_box])
    edit_btn.click(edit_approve, [args_editor], [result_display, approval_box])
    clear.click(lambda: ([], None, gr.update(visible=False)), None, [chatbot, interrupt_display, approval_box])

print("✅ Interface created")

## Step 10: Launch the App

### 📱 How to Use the Tabbed Interface

The app has **2 tabs**:

#### 1️⃣ **Chat Tab** (Main Interface)
- Send messages to the HR agent
- View conversation history
- Try example queries
- When approval is needed, you'll be notified to switch tabs

#### 2️⃣ **Approval Panel Tab** (Manager Actions)
- Review pending approval requests
- See tool name and arguments
- Three decision options:
  - **✅ Approve**: Execute as-is
  - **✏️ Edit & Approve**: Modify JSON arguments first
  - **❌ Reject**: Cancel with optional reason

### 🔄 Workflow:
1. **Chat Tab**: Send request (e.g., "Update salary for employee 101 to ₹150,000")
2. Agent responds: "Operation Paused - Go to Approval Panel tab"
3. **Switch to Approval Panel Tab**
4. Review the request details
5. Click **Approve**, **Edit & Approve**, or **Reject**
6. **Switch back to Chat Tab** to continue conversation

### ⚠️ Important Notes:
- **DO NOT** send new chat messages when approval is pending
- **ALWAYS** use the Approval Panel tab buttons for decisions
- The status bar shows if there's a pending approval
- After approval/rejection, you can continue chatting normally

---

In [ ]:
demo.launch(share=True, debug=True)